# CAMEL AI Multi Agent Example

In this example, we will use CamelAI to simulate tools! In this case, we will best determine how many shots it takes to beat an enemy with a blue shield in Apex Legeneds using melee only. The character "Pathfinder" from Apex Legends will answer.

First let's install the required packages for this example.

In [ ]:
%pip install camel-ai[all]
%pip install agentops

Next we import the necessary libraries

In [ ]:
import agentops
import os
from getpass import getpass
from dotenv import load_dotenv
from typing import List
from colorama import Fore

# Camel imports
from camel.agents.chat_agent import FunctionCallingRecord
from camel.models import ModelFactory
from camel.societies import RolePlaying
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated
from camel.toolkits import SearchToolkit, MathToolkit

Next, we'll set our API keys. There are several ways to do this, the code below is just the most foolproof way for the purposes of this notebook. It accounts for both users who use environment variables and those who just want to set the API Key here in this notebook.

[Get an AgentOps API key](https://agentops.ai/settings/projects)

1. Create an environment variable in a .env file or other method. By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Or...

2. Replace `<your_agentops_key>` below and pass in the optional `api_key` parameter to the AgentOps `init(api_key=...)` function. Remember not to commit your API key to a public repo!

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your openai key here>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your agentops key here>"

Now we will initialize our AgentOps client.

In [ ]:
agentops.init(default_tags=["camel", "multi-agent", "example"])

Let's start with setting our task prompt and setting our tools.

You can look at the link below to see all available tools:
https://docs.camel-ai.org/key_modules/tools.html

In [4]:
task_prompt = (
    "We are in the Apex Legends Games. Determine the amount of"
    "meele hits it will take to beat someone with a blue shield."
    "You should answer as if you are Pathfinder from the Apex Games."
)

tools = [
    *MathToolkit().get_tools(),
    *SearchToolkit().get_tools(),
]

We will now create our Camel AI session which is of [`RolePlaying`](https://docs.camel-ai.org/key_modules/society.html#roleplaying) type. Here we will set the assistant and user role names, as well as the model and tools for each agent.

In [ ]:
search_session = RolePlaying(
    assistant_role_name="Searcher",
    user_role_name="Pathfinder",
    assistant_agent_kwargs=dict(
        model=ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI,
            model_type=ModelType.GPT_4O_MINI,
        ),
        tools=tools,
    ),
    user_agent_kwargs=dict(
        model=ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI,
            model_type=ModelType.GPT_4O_MINI,
        ),
    ),
    task_prompt=task_prompt,
    with_task_specify=False,
)

Let's print out the Assistant System Message and User Task Prompt.

In [ ]:
print(
    Fore.GREEN
    + f"AI Assistant System Message:\n{search_session.assistant_sys_msg}\n"
)
print(Fore.BLUE + f"AI User System Message:\n{search_session.user_sys_msg}\n")

print(Fore.YELLOW + f"Original Task Prompt:\n{task_prompt}\n")
print(
    Fore.CYAN
    + "Specified Task Prompt:"
    + f"\n{search_session.specified_task_prompt}\n"
)
print(Fore.RED + f"Final Task Prompt:\n{search_session.task_prompt}\n")

Now we will initiate our Camel AI session and begin the chat loop. You can see that we have set the number of messages to 50. This is to prevent the session from running indefinitely.

In [ ]:
n = 0
input_msg = search_session.init_chat()
while n < 50:
    n += 1
    assistant_response, user_response = search_session.step(input_msg)

    if assistant_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI Assistant terminated. Reason: "
                f"{assistant_response.info['termination_reasons']}."
            )
        )
        break
    if user_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI User terminated. "
                f"Reason: {user_response.info['termination_reasons']}."
            )
        )
        break

    # Print output from the user
    print_text_animated(
        Fore.BLUE + f"AI User:\n\n{user_response.msg.content}\n"
    )

    # Print output from the assistant, including any function execution information
    print_text_animated(Fore.GREEN + "AI Assistant:")
    tool_calls: List[FunctionCallingRecord] = assistant_response.info[
        'tool_calls'
    ]
    for func_record in tool_calls:
        print_text_animated(f"{func_record}")
    print_text_animated(f"{assistant_response.msg.content}\n")

    if "CAMEL_TASK_DONE" in user_response.msg.content:
        break

    input_msg = assistant_response.msg

Awesome! We've successfully completed our session.

Now we will end the session with a success message. We can also end the session with a failure or indeterminate status. By default, the session will be marked as indeterminate.

In [ ]:
agentops.end_session("Success")

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)

Now if we look in the AgentOps dashboard, you will see a session recorded with the LLM calls and tool usage.